In [1]:
import json

with open(r"D:\fillable-10-21\fillable_assignments_2.json") as f:
    data = json.load(f)

In [2]:
import collections

ctr = collections.Counter(data.values())
print(ctr.most_common())

has_assignment = {k for k, v in data.items()}

[('generated', 609), ('ray', 101), ('notfillable', 56), ('combined', 51), ('dip', 12)]


In [3]:
# What files are available?
import pathlib

volume_files = {x for x in pathlib.Path(r"D:\fillable-10-21").glob(r"objects\*\*\fillable---*.obj")}
has_volume_file = {x.parts[-2] for x in volume_files}

In [4]:
from bddl.knowledge_base import *

Loading BDDL knowledge base... This may take a few seconds.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cgokmen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
objects = Object.all_objects()
obj_names = {o.name.split("-")[-1]: o.name for o in Object.all_objects()}


In [6]:
# Get all the objects that have a fillable synset
fillable_needing = {o.name.split("-")[-1] for o in objects if any(p.name == "fillable" for p in o.category.synset.properties) and o.ready}

In [7]:
fillable_missing = fillable_needing - has_assignment

In [8]:
print("Fillable needing: ", len(fillable_needing))
print("Fillable assigned:", len(has_assignment))
print("Fillable missing:", len(fillable_missing))

Fillable needing:  1031
Fillable assigned: 837
Fillable missing: 194


In [14]:
# Check if any non-fillable-needing objects have a fillable mesh
for vf in volume_files:
  if vf.parts[-2] not in fillable_needing:
    print(f"Non-fillable needing object {vf.parts[-2]} has a fillable mesh")


In [20]:
# Are there any directories that don't belong to fillables? Remove them
import shutil
if False:
  for directory in pathlib.Path(r"D:\fillable-10-21").glob(r"objects\*\*"):
    if directory.is_dir() and directory.parts[-1] not in fillable_needing:
      print(f"Removing {directory}")
      shutil.rmtree(directory)

In [23]:
# Who needs a fillable volume but has no candidates right now?
fillable_missing_no_candidates = fillable_needing - has_ray - has_dip
print("Fillable missing no candidates:", len(fillable_missing_no_candidates))
print(fillable_missing_no_candidates)

Fillable missing no candidates: 218
{'cwkvib', 'odmjdd', 'kxovsj', 'kieuxg', 'qkkjwp', 'qkgfln', 'glefdh', 'zeoukc', 'vhnlno', 'rywinu', 'wauyns', 'qnequy', 'qyuyjr', 'dylnro', 'xfmyyw', 'jasnzj', 'eupgpt', 'ujodgo', 'pyilfa', 'xzusqq', 'wbnkfk', 'spppps', 'msngax', 'ekjpdj', 'xnjqix', 'dnqekb', 'yxfueo', 'lleghp', 'coanpv', 'pkkgzc', 'ksgizx', 'rmmvcd', 'vxobmt', 'ntedfx', 'aewthq', 'tolupn', 'kkjiko', 'oyqdtz', 'rowulr', 'kiiamx', 'kmgltg', 'zuwvdo', 'ypdfrp', 'rntwkg', 'guobeq', 'kkmkbd', 'aakcyj', 'euzudc', 'wgcgia', 'ttmejh', 'iawoof', 'ilofmb', 'vwylob', 'akusda', 'lulzdz', 'cjmtvq', 'ahbhsd', 'xmjzos', 'ezsdil', 'eixyyn', 'xngcbz', 'dszchb', 'wxoxxg', 'xtdcau', 'dtaove', 'wengzf', 'oshwps', 'wranvj', 'lkomhp', 'msaevo', 'bnored', 'mbmbpa', 'wqgndf', 'ncgmqr', 'pbjodr', 'mwgceh', 'hxccge', 'todoxz', 'nhodax', 'tgrsui', 'immwzb', 'dsbcxl', 'cmdagy', 'lwjdmj', 'nbuspz', 'nrjump', 'dajebq', 'yvhmex', 'xqmipa', 'bgupna', 'ggbdlq', 'pzjecb', 'vtjwof', 'pkdnbu', 'vxmzmq', 'juwaoh', 'vb

In [11]:
print(sorted(obj_names[x] for x in fillable_missing))

['clothes_dryer-jasnzj', 'fuel_can-bfaqfe', 'shopping_basket-nedrsh']


In [18]:
# Are there any fillables that did not get processed?
print({o.name for o in objects if any(p.name == "fillable" for p in o.category.synset.properties) and o.name.split("-")[-1] not in data})

{'trash_can-jlawet', 'soda_cup-fsfsas', 'clothes_dryer-jasnzj'}


In [19]:
# Export a copy removing everything with no mesh selection
selected_only = {k: v for k, v in data.items() if v in {"combined", "ray", "dip"}}
with open("/scr/ig_pipeline/metadata/fillable_assignments_selected.json", "w") as f:
    json.dump(selected_only, f, indent=4)

In [20]:
# Group all the to-be-fixed objects by provider
grouped = collections.defaultdict(set)
fix_needing = {k for k, v in data.items() if v in ("fix", "changecollision")}
for obj in Object.all_objects():
    model = obj.name.split("-")[-1]
    if model not in fix_needing:
        continue
    grouped[obj.provider].add(obj.name)

for provider, objs in grouped.items():
    print(f"\n{provider}:")
    for obj in sorted(objs):
        print(f"  - {obj}")


objects/batch-05:
  - fuel_can-bfaqfe

scenes/grocery_store_half_stocked:
  - shopping_basket-nedrsh


In [11]:
not_fillable = {k for k, v in data.items() if v in {"notfillable"}}
synset_fillable_marked_notfillable = fillable_needing & not_fillable
print("\n".join(sorted([x.name for x in Object.all_objects() if x.name.split("-")[-1] in synset_fillable_marked_notfillable])))

In [12]:
fixorn = {k for k, v in data.items() if v in {"fixorn"}}
synset_fillable_marked_fixorn = fillable_needing & fixorn
print("\n".join(sorted([x.name for x in Object.all_objects() if x.name.split("-")[-1] in synset_fillable_marked_fixorn])))

bowl-fedafr
rice_cooker-qatgeb


In [13]:
# Remove the collision assignments of everything in the fix, fixorn and changecollision piles
import glob

import sys
sys.path.append("/scr/ig_pipeline")
        
from b1k_pipeline.utils import parse_name

if False:
    for file in glob.glob("/scr/ig_pipeline/cad/*/*/artifacts/collision_selection.json"):
        # Open each target's collision selection file
        with open(file, "r") as f:
            collision_selections = json.load(f)

        # Find which models are in the fix, fixorn and changecollision piles
        keys_to_remove = []
        for k, v in collision_selections.items():
            model_id = parse_name(k).group("model_id")
            if model_id in data and data[model_id] in {"fix", "fixorn", "changecollision"}:
                keys_to_remove.append(k)

        # Actually remove the collision assignments
        for k in keys_to_remove:
            del collision_selections[k]

        # Write the new collision selection file
        with open(file, "w") as f:
            json.dump(collision_selections, f, indent=4)

In [14]:
# Remove any non-fillable objects from the fillable assignment list
import shutil, glob
if False:
    for obj in objects:
        cat, mdl = obj.name.split("-")
        if mdl not in fillable_needing:
            if mdl in data:
                del data[mdl]
            volume_dirs = glob.glob(f"/scr/ig_pipeline/artifacts/parallels/fillable_volumes/objects/*/{mdl}")
            for vd in volume_dirs:
                shutil.rmtree(vd)

    with open("/scr/ig_pipeline/metadata/fillable_assignments.json", "w") as f:
        json.dump(data, f, indent=4)

In [13]:
# Remove generated meshes for the "fix", "manual", "fixorn" and "changecollision" objects and try again.
import shutil, glob
if True:
    for obj in objects:
        cat, mdl = obj.name.split("-")
        if mdl not in data or data[mdl] not in {"combined", "ray", "dip"}:
            if mdl in data:
                del data[mdl]
            volume_dirs = glob.glob(f"/scr/ig_pipeline/artifacts/parallels/fillable_volumes/objects/*/{mdl}")
            for vd in volume_dirs:
                shutil.rmtree(vd)

    with open("/scr/ig_pipeline/metadata/fillable_assignments.json", "w") as f:
        json.dump(data, f, indent=4)

In [7]:
print("\n".join(f'"{x}",' for x in sorted(fillable_needing)))

"aakcyj",
"ackxiy",
"adciys",
"adiwil",
"adxzhe",
"aefcem",
"aegxpb",
"aeslmf",
"aewpzn",
"aewthq",
"ahbhsd",
"ahtzhp",
"ajzltc",
"akfjxx",
"akusda",
"amhlqh",
"ankfvi",
"apybok",
"arryyl",
"aspeds",
"atgnsc",
"avotsj",
"aynjhg",
"aysfhf",
"azoiaq",
"barzwx",
"bbduix",
"bbewjo",
"bbpraa",
"bdhvnt",
"bedkqu",
"belcml",
"bexgtn",
"bfaqfe",
"bgxzec",
"bnekjp",
"bnobdx",
"bnored",
"bojwlu",
"bovcqx",
"bpwjxr",
"bqpmsv",
"bsdexp",
"bsgybx",
"bupgpj",
"byzaxy",
"bzisss",
"bzsxgw",
"causya",
"cdmmwy",
"cdteyb",
"cdzyew",
"ceaeqf",
"cfdond",
"chjetk",
"cjmezk",
"cjmtvq",
"cjsbft",
"ckkwmj",
"ckxwea",
"cmdagy",
"coqeme",
"cpozxi",
"cprjvq",
"cqdioi",
"crlhmi",
"csanbr",
"csvdbe",
"cvdbum",
"cvyops",
"cwkvib",
"cydfkt",
"cypjlv",
"dafdgk",
"dalyim",
"dbprwc",
"dcleem",
"deudkt",
"dfjcsi",
"dhfqid",
"dhgtvg",
"dhkkfo",
"dhnxww",
"dhseui",
"dhwlaw",
"djgllo",
"dkxddg",
"dnqekb",
"dnvpag",
"dobgmu",
"dpxnlc",
"dtjmai",
"duugbb",
"dwspgo",
"eahqyq",
"eawgwj",
"ecqxgd",
"edfzlt",
"effbnc",
"efkgcw",


In [10]:
{obj_names[k] for k, v in data.items() if v in {"manual"}}

{'blender-dhfqid',
 'blender-xnjqix',
 'mixing_bowl-rgnjdt',
 'plate-xtdcau',
 'platter-csanbr',
 'rice_cooker-qatgeb',
 'shopping_basket-nedrsh',
 'wine_bottle-inkqch'}